In [1]:
import torch
import torch.nn as nn
from pathlib import Path
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from math import *
from random import gauss,seed
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import seaborn

In [2]:
from PythonModules.RESNET import RESNET, DensityEstimator

In [3]:
import sys 
sys.path.insert(0,'/home/mohit.kumargupta/deep_boltzmann')
from matplotlib import rcParams
import keras
import tensorflow as tf
from deep_boltzmann.models import ParticleDimer
from deep_boltzmann.networks.invertible import invnet, EnergyInvNet, create_RealNVPNet
from deep_boltzmann.sampling import GaussianPriorMCMC
from deep_boltzmann.sampling.latent_sampling import BiasedModel
from deep_boltzmann.sampling.permutation import HungarianMapper
from deep_boltzmann.util import load_obj, save_obj
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite

Using TensorFlow backend.
/home/mohit.kumargupta/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mohit.kumargupta/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mohit.kumargupta/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mohit.kumargupta/.local/lib/p

In [4]:
trajdict = np.load('output.npz')
import ast
#params = ast.literal_eval(str(trajdict['params']))
traj_closed_train = trajdict['traj_closed_train_hungarian']
traj_open_train = trajdict['traj_open_train_hungarian']
traj_closed_test = trajdict['traj_closed_test_hungarian']
traj_open_test = trajdict['traj_open_test_hungarian']
x = np.vstack([traj_closed_train, traj_open_train])
xval = np.vstack([traj_closed_test, traj_open_test])

In [5]:
class combineModel(nn.Module):
    def __init__(self,model1,model2,model3):
        super(combineModel,self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
    
    def forward(self,inp):
        out = self.model1(inp)
        out = torch.reshape(out,(out.shape[0],1,8,8))
        #print(out)
        o1 = out
        out = self.model2(out)
        o2 = out
        #out = 36*torch.softmax( torch.reshape(out,(out.shape[0],16*16)), 1)
        #print(out.shape[0])
        #out = torch.reshape(out,(out.shape[0],1,16,16))
        
        #print(out)
        out = self.model3(out)
        #out = 36*torch.softmax( torch.reshape(out,(out.shape[0],32*32)), 1)
        #out = torch.reshape(out,(out.shape[0],1,32,32))
        
        return out

In [6]:
class ConfigurationModel(nn.Module):
    
    def __init__(self,model1):
        super(ConfigurationModel,self).__init__()
        self.model1 = model1
        self.hidden_dim = 32*16
        self.out_dim = 72
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.fc1 = nn.Linear(32*32, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, 32*16)
        self.fc3 = nn.Linear(32*16, 16*16)
        #self.lstm = nn.LSTM(hidden_dim, output_dim)
        #self.fc3 = nn.Linear(self.hidden_dim*4*4, hidden_dim*4)
        self.fc4 = nn.Linear(16*16, self.out_dim)
    
    def forward(self,inp):
        out = self.model1(inp)
        out = out.reshape((inp.shape[0],32*32))
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.fc4(out)
        return out

In [7]:
batch_size = 500
train_set = np.vstack([traj_open_train[1000:81000 :2], traj_closed_train[1000:81000 :2] ] )

In [8]:
inputData = torch.from_numpy(train_set[:80000,:4]).type('torch.FloatTensor')
dataSet32 = torch.from_numpy( np.concatenate( (np.load('NewDataSet32.npy')[:40000], np.load('NewDataSet32.npy')[49500:89500] )) ).type('torch.FloatTensor')

In [9]:
dset = torch.cat((inputData,torch.reshape(dataSet32,(dataSet32.shape[0],32*32))),1 )

In [10]:
train_loader = torch.utils.data.DataLoader(dset.type('torch.FloatTensor').to('cuda'), batch_size=batch_size,shuffle= True)

In [11]:
model1 = DensityEstimator(4,64)
model2 = RESNET(18)
model3 = RESNET(18)

In [12]:
CombineModel = combineModel(model1,model2,model3)

In [13]:
CombineModel.load_state_dict(torch.load('NewCombineDensityEstimator4-64'))

<All keys matched successfully>

In [14]:
CombineModel.eval()

combineModel(
  (model1): DensityEstimator(
    (relu): LeakyReLU(negative_slope=0.2, inplace=True)
    (fc1): Linear(in_features=4, out_features=16, bias=True)
    (fc2): Linear(in_features=16, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=256, bias=True)
    (fc4): Linear(in_features=256, out_features=64, bias=True)
  )
  (model2): RESNET(
    (conv_input): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (relu): LeakyReLU(negative_slope=0.2, inplace=True)
    (residual): Sequential(
      (0): Residual_Block(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (in1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (relu): LeakyReLU(negative_slope=0.2, inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (in2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [15]:
conModel = ConfigurationModel(CombineModel.to('cuda')).to('cuda')

In [34]:
adOptimizer = optim.Adam(conModel.parameters(),lr =0.0000003)

In [17]:
boxes = []
sz =32
x = -3.25
while x< 3.25:
        y = -3.25
        while(y<3.25):
            boxes.append([x+3.25/sz, y +3.25/sz])
            y+= (3.25*2)/sz   
        x += (3.25*2)/sz
boxes = np.array(boxes)
boxes = np.tile(boxes,(36,1,1))

In [18]:
boxes = torch.from_numpy(boxes).type('torch.FloatTensor').to('cuda')

In [19]:
print(boxes.shape)

torch.Size([36, 1024, 2])


In [20]:
def myFunc(output,data32):
    global boxes
    boxing = boxes.unsqueeze(0).expand((output.shape[0],-1,-1,-1))
    sz = 32
    vol = (3.25*2/sz)**2
    #print( torch.reshape(output,(output.shape[0],36,2)).unsqueeze(2).shape)
    snap =  torch.reshape(output,(output.shape[0],36,2)).unsqueeze(2).expand( (-1,-1,1024,-1))
    dist = torch.norm(snap-boxing,dim = 3)
    #print(dist.shape)
    val =  vol * torch.exp(-0.5*torch.mul((dist/0.4), (dist/0.4)) )/ (2*3.14 * 0.16)
    #print(val.sum(axis=1).shape)
    dataSet32 = val.sum(axis=1).reshape(val.shape[0],32,32)
    #print(dataSet32.shape)
    #dataSet32 = torch.reshape(torch.from_numpy(dataSet32), (len(dataSet32),sz,sz) )
    return torch.sqrt( nn.MSELoss()(dataSet32,data32))

In [21]:
sampleBox = []

In [22]:
for i in range(6):
    for j in range(6):
        sampleBox.append(i-2.5)
        sampleBox.append(j-2.5)

In [23]:
sampleBox = torch.from_numpy(np.array(sampleBox)).type('torch.FloatTensor').to('cuda')

In [29]:
print(sampleBox.size())

torch.Size([72])


In [30]:
print(dataSet32.shape)

torch.Size([80000, 32, 32])


In [31]:
boxes.shape

torch.Size([36, 1024, 2])

In [28]:
Epochs = 100

In [24]:
conModel.load_state_dict(torch.load('conModel4-72'))
conModel.eval()

ConfigurationModel(
  (model1): combineModel(
    (model1): DensityEstimator(
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (fc1): Linear(in_features=4, out_features=16, bias=True)
      (fc2): Linear(in_features=16, out_features=64, bias=True)
      (fc3): Linear(in_features=64, out_features=256, bias=True)
      (fc4): Linear(in_features=256, out_features=64, bias=True)
    )
    (model2): RESNET(
      (conv_input): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (residual): Sequential(
        (0): Residual_Block(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (in1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (relu): LeakyReLU(negative_slope=0.2, inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        

In [ ]:
for epoch in range(Epochs):
    totalLoss = 0.0
    print(epoch)
    if((epoch+1)%2 == 0):
        torch.save(CombineModel.state_dict(), 'conModel4-72')
    for i,data in enumerate(train_loader,1):
        adOptimizer.zero_grad()
        out = conModel(data[:,:4])
        #print(out)
        #l1 = torch.sqrt( nn.MSELoss()(out,sampleBox.unsqueeze(0).expand(out.shape[0],-1)))
        #print(data.type(),out.type())
        l2 = myFunc(out, data[:,4:].reshape((data.shape[0],32,32)))
        loss = l2
        loss.backward()
        print(i,loss.item())
        totalLoss += loss.item()
        if(math.isnan(loss.item()) or math.isinf(loss.item())):
            break
        adOptimizer.step()
    print(totalLoss)

0
1 0.006835180334746838
2 0.006834129802882671
3 0.006822367664426565
4 0.006806529127061367
5 0.006830344442278147
6 0.006815469823777676
7 0.0068234894424676895
8 0.0068372879177331924
9 0.00677922647446394
10 0.006790398620069027
11 0.0068070609122514725
12 0.0067924573086202145
13 0.00678332569077611
14 0.00679463567212224
15 0.006753771565854549
16 0.0068559711799025536
17 0.006782745476812124
18 0.00683381874114275
19 0.00676791463047266
20 0.006808572448790073
21 0.006812112871557474
22 0.006843631621450186
23 0.00677460664883256
24 0.0068083335645496845
25 0.006804104428738356
26 0.0067963688634335995
27 0.006853276863694191
28 0.006783377844840288
29 0.006813926622271538
30 0.006820396054536104
31 0.00680934265255928
32 0.006821549963206053
33 0.006832682061940432
34 0.006799568887799978
35 0.0067926011979579926
36 0.006823358125984669
37 0.006806653458625078
38 0.006839758716523647
39 0.006817456800490618
40 0.006832600105553865
41 0.006811166647821665
42 0.00683243433013558

16 0.006764333229511976
17 0.0067990925163030624
18 0.006788359954953194
19 0.006847133859992027
20 0.006744300480931997
21 0.006826523691415787
22 0.006807619705796242
23 0.0067845783196389675
24 0.006795965135097504
25 0.006848758086562157
26 0.006779490504413843
27 0.006868517957627773
28 0.0068385121412575245
29 0.00678291404619813
30 0.006801251322031021
31 0.006796745117753744
32 0.006786711513996124
33 0.0067632244899868965
34 0.0067930687218904495
35 0.00676322914659977
36 0.0068206945434212685
37 0.006863077636808157
38 0.006819081958383322
39 0.006813338026404381
40 0.0068419091403484344
41 0.006763922516256571
42 0.006816681940108538
43 0.0068080369383096695
44 0.006806930061429739
45 0.006792871281504631
46 0.00680750235915184
47 0.006816635373979807
48 0.006803298834711313
49 0.006804979871958494
50 0.006834215484559536
51 0.0068362089805305
52 0.006780686788260937
53 0.006826345808804035
54 0.006796017289161682
55 0.006854251027107239
56 0.006759870331734419
57 0.00683314

31 0.006803614553064108
32 0.006848365068435669
33 0.006820544600486755
34 0.0068283686414361
35 0.006799772847443819
36 0.0068038394674658775
37 0.00681979488581419
38 0.006777769885957241
39 0.006753569468855858
40 0.006797569803893566
41 0.006813552230596542
42 0.006799459923058748
43 0.006813306827098131
44 0.006772635970264673
45 0.006810360588133335
46 0.00681571988388896
47 0.006864698603749275
48 0.006794671528041363
49 0.006792159751057625
50 0.006821164395660162
51 0.006835899781435728
52 0.006798860616981983
53 0.006815043278038502
54 0.006822734139859676
55 0.006812332198023796
56 0.006790793035179377
57 0.006795748136937618
58 0.006855166517198086
59 0.0068018012680113316
60 0.006805954501032829
61 0.006838124245405197
62 0.006801747716963291
63 0.006761359516531229
64 0.006791317835450172
65 0.006847304757684469
66 0.006855903193354607
67 0.006788494531065226
68 0.00681258924305439
69 0.0067154718562960625
70 0.006779453717172146
71 0.006800604052841663
72 0.0067936084233

46 0.00688976002857089
47 0.006816240027546883
48 0.006815439555794001
49 0.006780780851840973
50 0.006842592731118202
51 0.006797791924327612
52 0.0068013472482562065
53 0.006820398382842541
54 0.006808368023484945
55 0.006798091344535351
56 0.006793047301471233
57 0.006848933175206184
58 0.006787582766264677
59 0.006746012717485428
60 0.006835720967501402
61 0.006803502328693867
62 0.006835410837084055
63 0.006783386692404747
64 0.006807295139878988
65 0.006805682089179754
66 0.006826735567301512
67 0.006761892698705196
68 0.006786182522773743
69 0.006837204098701477
70 0.0067711761221289635
71 0.0068143936805427074
72 0.00679028732702136
73 0.006819423753768206
74 0.006758294999599457
75 0.006746855564415455
76 0.006791998166590929
77 0.006781811360269785
78 0.006824680604040623
79 0.006814289838075638
80 0.006840937305241823
81 0.006798281334340572
82 0.006782936863601208
83 0.00681107584387064
84 0.006793269421905279
85 0.006824128795415163
86 0.00676985178142786
87 0.006796027533

61 0.006775722838938236
62 0.006843800190836191
63 0.0068454924039542675
64 0.006813923362642527
65 0.006805043201893568
66 0.006788667291402817
67 0.006838134489953518
68 0.00680135702714324
69 0.0068521262146532536
70 0.006809943821281195
71 0.006784651428461075
72 0.006866618990898132
73 0.0068167271092534065
74 0.006800160277634859
75 0.006789856590330601
76 0.006812639068812132
77 0.006769513711333275
78 0.006815899163484573
79 0.006776593625545502
80 0.006829104386270046
81 0.006745794788002968
82 0.006829261314123869
83 0.006741061341017485
84 0.0068171024322509766
85 0.0067646680399775505
86 0.006811328697949648
87 0.006792617030441761
88 0.006760603282600641
89 0.006744766142219305
90 0.006829232443124056
91 0.006793862208724022
92 0.006859412416815758
93 0.006804298143833876
94 0.0068299975246191025
95 0.006848361808806658
96 0.006771095562726259
97 0.00686334865167737
98 0.0068220640532672405
99 0.006851156707853079
100 0.006822596769779921
101 0.006793450564146042
102 0.006

...

33 0.006820833310484886
34 0.00678561395034194
35 0.006767410784959793
36 0.006877552252262831
37 0.006877771113067865
38 0.006731986999511719
39 0.006753130815923214
40 0.006811399478465319
41 0.006862149573862553
42 0.006756810937076807
43 0.0068437703885138035
44 0.006799605675041676
45 0.006845911033451557
46 0.006848483346402645
47 0.006768560502678156
48 0.006818724796175957
49 0.006827186327427626
50 0.006789615843445063
51 0.006769315805286169
52 0.0067365230061113834
53 0.006732667330652475
54 0.006800568196922541
55 0.00683760829269886
56 0.006787087302654982
57 0.006810098420828581
58 0.00681382417678833
59 0.006755264010280371
60 0.006772583816200495
61 0.0068351952359080315
62 0.006781070958822966
63 0.006763847079128027
64 0.0067652128636837006
65 0.006759838201105595
66 0.006766934413462877
67 0.006843205075711012
68 0.006839029025286436
69 0.006785966921597719
70 0.006835618056356907
71 0.006790672428905964
72 0.0068229674361646175
73 0.0067979455925524235
74 0.00684235

48 0.00680621387436986
49 0.006798991467803717
50 0.0068387314677238464
51 0.0068152425810694695
52 0.006765185855329037
53 0.006827560719102621
54 0.0068341526202857494
55 0.006819587666541338
56 0.006812249775975943
57 0.006838543340563774
58 0.006839781533926725
59 0.00677215401083231
60 0.006765320431441069
61 0.006785003002732992
62 0.0067990366369485855
63 0.006785503588616848
64 0.006768415216356516
65 0.006804765202105045
66 0.006788530386984348
67 0.006834580097347498
68 0.006784556433558464
69 0.006825046148151159
70 0.006827351171523333
71 0.006789616774767637
72 0.006766426842659712
73 0.006766162347048521
74 0.006770791485905647
75 0.006779483053833246
76 0.006797452922910452
77 0.006842195522040129
78 0.0067908079363405704
79 0.006794326938688755
80 0.006788794882595539
81 0.006797984708100557
82 0.006803774740546942
83 0.006805288605391979
84 0.00677461177110672
85 0.006785641890019178
86 0.006769340019673109
87 0.006746599450707436
88 0.006827975157648325
89 0.006810559

63 0.006837000604718924
64 0.0068220836110413074
65 0.006763685494661331
66 0.006815687753260136
67 0.006766993086785078
68 0.006795667111873627
69 0.006785689853131771
70 0.006814365740865469
71 0.006735175848007202
72 0.006792925763875246
73 0.006777362432330847
74 0.006760099902749062
75 0.006815360859036446
76 0.0067971511743962765
77 0.006827524863183498
78 0.006797554902732372
79 0.006813623011112213
80 0.0068161990493535995
81 0.006831249687820673
82 0.006830985192209482
83 0.00680566905066371
84 0.00680441502481699
85 0.00675544235855341
86 0.006778886541724205
87 0.00680143479257822
88 0.006833580322563648
89 0.006690198089927435
90 0.006764078047126532
91 0.006779683753848076
92 0.006795202381908894
93 0.006759011652320623
94 0.006781949661672115
95 0.006778409704566002
96 0.006799939554184675
97 0.006820762064307928
98 0.006798656191676855
99 0.0068118637427687645
100 0.006780948024243116
101 0.006812401115894318
102 0.006769559811800718
103 0.006833046209067106
104 0.006803

79 0.006775201763957739
80 0.006805071607232094
81 0.006759156938642263
82 0.006832003127783537
83 0.0067800115793943405
84 0.0067617581225931644
85 0.006873745936900377
86 0.006834658794105053
87 0.006873054429888725
88 0.006781444419175386
89 0.006809186190366745
90 0.006823413539677858
91 0.006797944661229849
92 0.006826391443610191
93 0.00680165272206068
94 0.0067640142515301704
95 0.006787391845136881
96 0.006825868971645832
97 0.006799223367124796
98 0.006834710482507944
99 0.006767123471945524
100 0.006803563795983791
101 0.006818415597081184
102 0.0067235748283565044
103 0.006814292632043362
104 0.006774041336029768
105 0.006843437906354666
106 0.006764513906091452
107 0.006717045325785875
108 0.0068071698769927025
109 0.006772533990442753
110 0.0068351589143276215
111 0.006810316815972328
112 0.00681680254638195
113 0.006789875216782093
114 0.006809041369706392
115 0.00679527223110199
116 0.006841255817562342
117 0.006805016193538904
118 0.006864976137876511
119 0.006769722327

94 0.006771374493837357
95 0.006786912679672241
96 0.006732383277267218
97 0.006841753143817186
98 0.006798240821808577
99 0.006830104626715183
100 0.006826127879321575
101 0.006806759629398584
102 0.006820221431553364
103 0.006830186583101749
104 0.006755292881280184
105 0.00680652167648077
106 0.006765571888536215
107 0.006785657722502947
108 0.006779081653803587
109 0.0067841410636901855
110 0.006808043457567692
111 0.00680268881842494
112 0.006838464643806219
113 0.006782193202525377
114 0.0068314652889966965
115 0.006814064458012581
116 0.006838000379502773
117 0.006804614793509245
118 0.00681851664558053
119 0.006716356612741947
120 0.00674431724473834
121 0.006813159212470055
122 0.0068415249697864056
123 0.006777509115636349
124 0.006756232585757971
125 0.006814445368945599
126 0.006864562164992094
127 0.0068132635205984116
128 0.006795735098421574
129 0.0068398029543459415
130 0.0068264976143836975
131 0.006766483187675476
132 0.0068412418477237225
133 0.0067670950666069984
13

In [ ]:
torch.save( conModel.state_dict(), 'conModel34')

In [33]:
torch.save( conModel.state_dict(), 'conModel4-72')

In [42]:
model = ParticleDimer()

In [43]:
Energy = None

In [41]:
for i,data in enumerate(train_loader,1):
        out = conModel(data[:,:4])
        final = torch.cat( (data[:,:4],out),1).to('cpu')
        #print(final[10])
        
        if(Energy is None):
            Energy = model.energy(final.to('cpu').detach().numpy() )
        else: 
            Energy = np.concatenate((Energy, model.energy(final.to('cpu').detach().numpy())) )
        #print(model.energy(final.to('cpu').detach().numpy()) )
        #break

NameError: name 'Energy' is not defined

In [ ]:
print(Energy)

In [ ]:
plt.hist(Energy, bins=500, histtype= 'stepfilled')

In [ ]:
plt.hist(Energy, bins=100,cumulative= True, histtype= 'bar')